# ML Assignment 2 & 3   
# Paul Thillen, Louis-Philippe Noël
# 3. PRACTICAL PART (40 pts) : Neural network implementation and experiments

## Numerically stable softmax. 
You will need to compute a numerically
stable softmax. Refer to posted readings to see how to do this. Start by
writing the expression for a single vector, then adapt it for a mini-batch
of examples stored in a matrix.


In [24]:
%pylab inline
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.io
import gzip,pickle
import copy
import math

Populating the interactive namespace from numpy and matplotlib


C:\Users\Paul Thillen\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'copy', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


#Load data

In [25]:
#two moons 
two_moons = np.loadtxt(open('2moons.txt','r'))
#mnist
f=gzip.open('mnist.pkl.gz')
mnist=pickle.load(f)

#Utilities

In [26]:
def softmax(x):
    maximum = np.max(x, axis=-1, keepdims=True)
    top = np.exp(x - maximum)
    bottom = np.sum(top, axis=-1, keepdims=True)
    return top/bottom

#test
#A= array([[ 1.,  3.,  4.,  5.,  6.],
#       [ 4.,  5.,  6.,  3.,  3.],
#       [ 1.,  3.,  4.,  5.,  6.],
#       [ 4.,  5.,  6.,  3.,  3.]])
#softmax(A)

In [27]:
def uniforme(a,b):
    return(a+(b-a)*np.random.rand()) 

## Parameter initialization. 
As you know, it is necessary to randomly
initialize the parameters of your neural network (trying to avoid symme-
try and saturating neurons, and ideally so that the pre-activation lies in
the bending region of the activation function so that the overall networks
acts as a non linear function). We suggest that you sample the weights
of a layer from a uniform distribution in [ -1/sqrt(n_c), 1/sqrt(n_c) ], 
where n c is the number of inputs for this layer (changing from one layer to the other).
Biases can be initialized at 0. Justify any other initialization method.

## fprop
fprop will compute the forward progpagation i.e. step by step computation from the input to the output and the cost of the activations of each layer.

## bprop
bprop will use the computed activations by fprop and does
the backpropagation of the gradients from the cost to the input following
precisely the steps derived in part 2.

## Finite difference gradient check. We can estimate the gradient nu-
merically using the finite difference method. You will implement this estimate as a tool to check your gradient computation. To do so, calculate the value of the loss function for the current parameter values (for a single example or a mini batch). Then for each scalar parameter θ k , change the
parameter value by adding a small perturbation (10^−6 < ε < 10^−4 )
and calculate the new value of the loss (same example or minibatch),
then set the value of the parameter back to its original value. The partial
derivative with respect to this parameter is estimated by dividing the
change in the loss function by ε. The ratio of your gradient computed
by backpropagation and your estimate using finite difference should be
between 0.99 and 1.01.

All in one : Class

In [35]:
class Model:

    def plot_function(self, train_data, title):
        plt.figure()
        d1 = train_data[train_data[:, -1] > 0]
        d2 = train_data[train_data[:, -1] == 0]
        plt.scatter(d1[:, 0], d1[:, 1], c='b', label='classe 1')
        plt.scatter(d2[:, 0], d2[:, 1], c='g', label='classe 0')
        #x = np.linspace(np.min(train_data[:, 0]) - 0.5,
        #                np.max(train_data[:, 0]) + 0.5,
        #               100)
        #y = -(self.w2[0]*x + self.b2 - .5)/self.w2[1]
        #plt.plot(x, y, c='r', lw=2, label='y = -(w1*x + b1)/w2')
        plt.xlim(np.min(train_data[:, 0]) - 0.5, np.max(train_data[:, 0]) + 0.5)
        plt.ylim(np.min(train_data[:, 1]) - 0.5, np.max(train_data[:, 1]) + 0.5)
        plt.grid()
        plt.legend(loc='lower right')
        plt.title(title)
        plt.show()
        
        def plot_function_classifieur(self, train_data, title):
            if len(train_cols) == 2:
                # Surface de decision
                les_sorties = self.compute_predictions(train_data)
                train_classes_pred = np.argmax(les_sorties)
                plt.scatter(train_set[:,0],train_set[:,1],c=train_classes_pred)
                plt.scatter(test_set[:,0],test_set[:,1],c=classes_pred)
                plt.show()
                plt.scatter(train_set[:,0],train_set[:,1],c=train_set[:,-1])
                plt.scatter(test_set[:,0],test_set[:,1],c=test_labels)
                plt.show()
            else:
                print 'Trop de dimensions (', len(train_cols),') pour pouvoir afficher la surface de decision'

In [29]:
class neural_net(Model):
    
    def __init__(self, nc, m, k):
        #nc : number of neurons in the hidden layer
        #m : number of classes
        #k : size of mini-batches to use
        self.nc = nc
        self.m = m
        self.k = k
        
        
    def train(self,train_data, max_iter = 2, mu = 0.02, lamba = 0):
        self.lamba = lamba
        #print("x shape",self.x.shape)
        self.train_data = train_data
        #self.b1=np.zeros((self.nc,self.train_data.shape[1]-1))
        #self.b2=np.zeros((self.m,self.train_data.shape[1]-1))
        self.b1=np.zeros((self.nc,1))
        self.b2=np.zeros((self.m,1))
        self.nc1 = self.train_data.shape[1]-1
        self.nc2 = self.nc
        self.w1 = np.multiply(np.ones((self.nc,self.nc1)),uniforme(-1/sqrt(self.nc1),1/sqrt(self.nc1)))
        #print(self.nc,self.nc1)
        self.w2 = np.multiply(np.ones((self.m,self.nc)),uniforme(-1/sqrt(self.nc2),1/sqrt(self.nc2)))
        print("w1:",self.w1)
        print("w2:",self.w2)
        print("b1:",self.b1)
        print("b2:",self.b2)
        
        #gradient descent
        for i in range(max_iter):
            for j in range(int(math.ceil(train_data.shape[0]/float(self.k)))):
                print("intervalle j",int(math.ceil(train_data.shape[0]/self.k)))
                print("j",j)
                #set x and y according to mini-batch size
                self.x = np.transpose(train_data[j*self.k:(j+1)*self.k,:-1])
                self.y = np.transpose(train_data[j*self.k:(j+1)*self.k,-1])
                print("x",self.x)
                #calculate gradients with this minibatch
                self.fprop()
                self.bprop()
                
                #Calcul de la moyenne des gradients de tous les exemples de la mini-batch
                self.grad_w1 = np.mean(self.grad_w1, axis=0)
                self.grad_w2 = np.mean(self.grad_w2, axis=0)
                self.grad_b1 = np.mean(self.grad_b1, axis=0)
                self.grad_b2 = np.mean(self.grad_b2, axis=0)
                
                #mise à jour des paramètres
                self.w1 -= mu * self.grad_w1_el
                self.b1 -= mu * self.grad_b1
                self.w2 -= mu * self.grad_w2_el
                self.b2 -= mu * self.grad_b2
                
                #to remove
                print("w1:",self.w1)
                print("b1:",self.b1)
                print("w2:",self.w2)
                print("b2:",self.b2)
        
    def fprop(self):
        #ha : activation des neurones de la couche cachée
        self.ha= np.add(np.dot(self.w1,self.x),self.b1)
        #print(np.dot(self.w1,self.x))
        #print("ha:",self.ha)
        #print("b1:",self.b1, self.b1.shape)
        print("ha", self.ha.shape)
        #hs : sortie des neurones de la couche cachée
        self.hs= np.maximum(0,self.ha)
        print("hs", self.hs.shape)
        print("w2", self.w2.shape)
        #oa : activation des neurones de la couche de sortie
        self.oa= np.add(np.dot(self.w2,self.hs),self.b2)
        print("oa", self.oa.shape)
        #os : sortie des neurones de la couche cachée
        self.os=softmax(self.oa)
        print("os", self.os.shape)
        #L : fonction de coût
        self.L= -np.log(self.os)
        print("L", self.L.shape)
        #print(self.L)
        return self.L
    
    def compute_predictions(self, test_data):
        x_2 = test_data[:,:-1]
        ha = np.add(np.dot(self.w1,x_2),self.b1)
        hs= np.maximum(0,ha)
        oa= np.add(np.dot(self.w2,hs),self.b2)
        os=softmax(oa)
        return os
    
    def bprop(self):
        #grad_oa : gradient de la fonction d'activation de la couche de sortie par rapport à L
        onehot=np.zeros((self.m,self.x.shape[1]))
        onehot[np.int_(self.y)-1]=1
        self.grad_oa = self.os-onehot
        print("grad oa",self.grad_oa.shape)
        #grad_w2 et grad_b2
        self.grad_w2 = np.dot(self.grad_oa,np.transpose(self.hs))
        print("grad w2",self.grad_w2.shape)
        self.grad_b2 = np.sum(self.grad_oa, axis=1, keepdims=True)
        print("grad b2",self.grad_b2.shape)
        #grad_hs
        self.grad_hs = np.dot(np.transpose(self.w2),self.grad_oa)
        print("grad hs",self.grad_hs.shape)
        #grad_ha
        def I_ha(x):
            #x is a vector
            y=np.zeros((len(x),1))
            for i in range(0, len(x)):
                    if x[i,0]>0:
                        y[i,0] = 1
                    else:
                        y[i,0]=0
            return y

        self.grad_ha = np.multiply(self.grad_hs,I_ha(self.ha))
        print("grad ha",self.grad_ha.shape)
        #grad_w1 et grad_b1
        self.grad_w1 = np.dot(self.grad_ha,np.transpose(self.x))
        print("grad w1",self.grad_w1.shape)
        self.grad_b1 = np.sum(self.grad_ha, axis=1, keepdims=True)
        print("grad b1",self.grad_b1.shape)
        #grad_x
        self.grad_x = np.dot(np.transpose(self.w1),self.grad_ha)
        print("grad x",self.grad_x.shape)
        #elastic regularization
        self.grad_w2_el = self.grad_w2-self.lamba*(2*self.w2+np.sign(self.w2))
        print("grad w2 el",self.grad_w2_el.shape)
        self.grad_w1_el = self.grad_w1-self.lamba*(2*self.w1+np.sign(self.w1))
        print("grad w1 el",self.grad_w1_el.shape)
        
    #check if slightly changed parameters yield a similar gradient
    def finite_check(self):
        e = 0.00002
        #copier le réseau actuel et 
        self.net_copy = copy.copy(self)
        self.net_copy.w1 = np.add(self.w1,e)
        self.net_copy.w2 = np.add(self.w2,e)
        self.net_copy.b1 = np.add(self.b1,e)
        self.net_copy.b2 = np.add(self.b2,e)
        #gradients avec nouvelles valeurs des paramètres
        self.net_copy.fprop()
        self.net_copy.bprop()
        #check difference
        ratio_w1 = np.divide(self.grad_w1_el,self.net_copy.grad_w1_el)
        #print ratio_w1
        ratio_w2 = np.divide(self.grad_w2_el,self.net_copy.grad_w2_el)
        #print ratio_w2
        ratio_b1 = np.divide(self.grad_b1,self.net_copy.grad_b1)
        #print ratio_b1
        ratio_b2 = np.divide(self.grad_b2,self.net_copy.grad_b2)
        #print ratio_b2
        ratios = [ratio_w1,ratio_w2,ratio_b1,ratio_b2]
        #sortie
        for ratio in ratios:
            if (ratio<0.99).any() or (ratio>1.01).any():
                return False
        return True

 ## Size of the mini batches. 
We ask that your computation and gradient descent is done on minibatches (as opposed to the whole training set)
with adjustable size using a hyperparameter K. In the minibatch case,
we do not manipulate a single input vector, but rather a batch of input
vectors grouped in a matrix (that will give a matrix representation at
each layer, and for the input). In the case where the size is one, we obtain
an equivalent to the stochastic gradient. Given that numpy is eﬃcient on
matrix operations, it is more eﬃcient to perform computations on a whole
minibatch. It will greatly impact the execution time.

1. As a beginning, start with an implementation that computes the gradients
for a single example, and check that the gradient is correct using the ﬁnitediﬀerence method described above.

In [30]:
#exemple x : 3 neurones cachés, 2 dimensions, classe = 2
x= np.array([[1.2084724, 0.39429077, 1.]])
net1 = neural_net(2,2,100)
net1.train(x)
net1.fprop()
net1.bprop()
net1.finite_check()

('w1:', array([[-0.35818985, -0.35818985],
       [-0.35818985, -0.35818985]]))
('w2:', array([[-0.59877065, -0.59877065],
       [-0.59877065, -0.59877065]]))
('b1:', array([[ 0.],
       [ 0.]]))
('b2:', array([[ 0.],
       [ 0.]]))
('intervalle j', 0)
('j', 0)
('x', array([[ 1.2084724 ],
       [ 0.39429077]]))
('ha', (2L, 1L))
('hs', (2L, 1L))
('w2', (2L, 2L))
('oa', (2L, 1L))
('os', (2L, 1L))
('L', (2L, 1L))
('grad oa', (2L, 1L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 1L))
('grad ha', (2L, 1L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 1L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.35818985, -0.35818985],
       [-0.35818985, -0.35818985]]))
('b1:', array([[ 0.],
       [ 0.]]))
('w2:', array([[-0.59877065, -0.59877065],
       [-0.59877065, -0.59877065]]))
('b2:', array([[-0.01],
       [-0.01]]))
('intervalle j', 0)
('j', 0)
('x', array([[ 1.2084724 ],
       [ 0.39429077]]))
('ha', (2L, 1L))
('hs', (2L, 1L))
('

True

-2. Display the gradients for both methods (direct computation and ﬁnite
difference) for a small network (e.g. d = 2 and d h = 2) with random
weights and for a single example.

In [31]:
#exemple x : 2 neurones cachés, 2 dimensions, classe = 2
print("Gradient de x avec Direct Computation : ",net1.grad_x)
print("Gradient de x avec Finite Check : ",net1.net_copy.grad_x)

('Gradient de x avec Direct Computation : ', array([[ 0.],
       [ 0.]]))
('Gradient de x avec Finite Check : ', array([[ 0.],
       [ 0.]]))


-3. Add a hyperparameter for the minibatch size K to allow compute the
gradients on a minibatch of K examples (in a matrix), by looping over
the K examples (this is a small addition to your previous code).

In [32]:
#exemple x : 2 neurones cachés, 2 dimensions, classe = 2
x= two_moons[:5,:]
net3 = neural_net(3,2,2)
net3.train(x,2,0.5,0.5)

('w1:', array([[ 0.30977752,  0.30977752],
       [ 0.30977752,  0.30977752],
       [ 0.30977752,  0.30977752]]))
('w2:', array([[-0.06363582, -0.06363582, -0.06363582],
       [-0.06363582, -0.06363582, -0.06363582]]))
('b1:', array([[ 0.],
       [ 0.],
       [ 0.]]))
('b2:', array([[ 0.],
       [ 0.]]))
('intervalle j', 2)
('j', 0)
('x', array([[ 0.36540813, -1.2084724 ],
       [-1.7957237 ,  0.39429077]]))
('ha', (3L, 2L))
('hs', (3L, 2L))
('w2', (2L, 3L))
('oa', (2L, 2L))
('os', (2L, 2L))
('L', (2L, 2L))
('grad oa', (2L, 2L))
('grad w2', (2L, 3L))
('grad b2', (2L, 1L))
('grad hs', (3L, 2L))
('grad ha', (3L, 2L))
('grad w1', (3L, 2L))
('grad b1', (3L, 1L))
('grad x', (2L, 2L))
('grad w2 el', (2L, 3L))
('grad w1 el', (3L, 2L))
('w1:', array([[ 0.71466628,  0.71466628],
       [ 0.71466628,  0.71466628],
       [ 0.71466628,  0.71466628]]))
('b1:', array([[ 0.],
       [ 0.],
       [ 0.]]))
('w2:', array([[-0.34545374, -0.34545374, -0.34545374],
       [-0.34545374, -0.34545374,

-4. Display the gradients for both methods (direct computation and finite
difference) for a small network (e.g. d = 2 and dh = 2) with random
weights and for a minibatch with 10 examples (you can use examples from
both classes from the dataset 2 moons).

In [33]:
#exemple x : 2 neurones cachés, 2 dimensions, classe = 2
x= two_moons[:50,:]
net3 = neural_net(2,2,10)
net3.train(x,2,0.5)
net3.finite_check()
print("Gradient de x avec Direct Computation : ",net3.grad_x)
print("Gradient de x avec Finite Check : ",net3.net_copy.grad_x)

('w1:', array([[-0.25245831, -0.25245831],
       [-0.25245831, -0.25245831]]))
('w2:', array([[-0.17309233, -0.17309233],
       [-0.17309233, -0.17309233]]))
('b1:', array([[ 0.],
       [ 0.]]))
('b2:', array([[ 0.],
       [ 0.]]))
('intervalle j', 5)
('j', 0)
('x', array([[ 0.36540813, -1.2084724 ,  0.38618144, -0.63657005, -0.80009178,
         0.39643093,  0.26187853,  0.46829609,  0.34545833,  0.53459357],
       [-1.7957237 ,  0.39429077,  1.0483956 , -1.6521227 , -0.71212892,
        -1.8936056 ,  1.8775139 ,  1.8391074 , -1.0043363 ,  2.1266762 ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.26647756, -0.271748  ],
       [-0.26647756, -0.271748  ]]))
('b1:', array([[-1

('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.26647756, -0.271748  ],
       [-0.26647756, -0.271748  ]]))
('b1:', array([[-1.55783099],
       [-1.55783099]]))
('w2:', array([[ 0.76537202,  0.76537202],
       [ 0.76537202,  0.76537202]]))
('b2:', array([[ 45.],
       [ 45.]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('Gradient de x avec Direct Computation : ', array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]))


-5. Train your neural network using gradient descent on the dataset of the
two moons. Plot the decision regions for several different values of the
hyperparameters (weight decay, number of hidden units, early stopping)
so as to illustrate their effect on the capacity of the model.

In [34]:
x= two_moons[0:800,:]
test = two_moons[801:,:]
net3 = neural_net(2,2,10)
net3.train(x)
net3.plot_function_classifieur(test,"Graphic")

('w1:', array([[-0.05693186, -0.05693186],
       [-0.05693186, -0.05693186]]))
('w2:', array([[-0.22801055, -0.22801055],
       [-0.22801055, -0.22801055]]))
('b1:', array([[ 0.],
       [ 0.]]))
('b2:', array([[ 0.],
       [ 0.]]))
('intervalle j', 80)
('j', 0)
('x', array([[ 0.36540813, -1.2084724 ,  0.38618144, -0.63657005, -0.80009178,
         0.39643093,  0.26187853,  0.46829609,  0.34545833,  0.53459357],
       [-1.7957237 ,  0.39429077,  1.0483956 , -1.6521227 , -0.71212892,
        -1.8936056 ,  1.8775139 ,  1.8391074 , -1.0043363 ,  2.1266762 ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.05780363, -0.05854877],
       [-0.05780363, -0.05854877]]))
('b1:', array([[-

('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.09687073, -0.09772284],
       [-0.09687073, -0.09772284]]))
('b1:', array([[-0.15908869],
       [-0.15908869]]))
('w2:', array([[-0.21166263, -0.21166263],
       [-0.21166263, -0.21166263]]))
('b2:', array([[ 1.8],
       [ 1.8]]))
('intervalle j', 80)
('j', 10)
('x', array([[-1.5591325 , -0.39223501,  1.2357844 , -0.03606903,  0.11880725,
         1.7639149 , -1.2501597 , -0.97996157, -0.40662168, -0.47875676],
       [ 1.1176319 , -0.78796659,  0.74808358,  1.0340178 ,  0.54364242,
        -0.67916949,  1.0779969 ,  1.7613064 , -0.89232809,  2.0998766 ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad o

       [ 3.78]]))
('intervalle j', 80)
('j', 21)
('x', array([[ 1.2758539 ,  0.82209846, -0.51786667, -0.25858107, -0.65202305,
         1.3055621 ,  0.70538594, -0.22333161, -0.707287  , -0.35085422],
       [ 0.25620323, -1.7886765 , -0.56877807,  0.93149055,  2.091376  ,
        -0.08183684, -0.87334698, -1.2612688 ,  1.7341582 , -2.0578667 ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.07427778, -0.12339075],
       [-0.07427778, -0.12339075]]))
('b1:', array([[-0.23245018],
       [-0.23245018]]))
('w2:', array([[-0.19990753, -0.19990753],
       [-0.19990753, -0.19990753]]))
('b2:', array([[ 3.96],
       [ 3.96]]))
('intervalle j', 80)
('j', 22)
('x', array([[-1.0308217 , 

('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19453422, -0.19453422],
       [-0.19453422, -0.19453422]]))
('b2:', array([[ 5.58],
       [ 5.58]]))
('intervalle j', 80)
('j', 31)
('x', array([[-1.1518855 ,  1.5370276 ,  1.2277893 , -0.73314337, -0.33198327,
        -0.36267308,  1.3060306 , -1.0485254 , -0.66622432,  0.99496833],
       [ 0.84063508,  0.01726746, -1.2126374 ,  0.87005352, -0.7255234 ,
        -1.9883846 , -0.38192666, -0.61712901, -0.7229541 , -1.8742351 ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))


('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19453422, -0.19453422],
       [-0.19453422, -0.19453422]]))
('b2:', array([[ 7.56],
       [ 7.56]]))
('intervalle j', 80)
('j', 42)
('x', array([[-0.0666715, -1.34127  ,  1.18544  ,  0.486935 ,  0.278286 ,
         1.52429  ,  1.32507  , -0.403139 , -0.765354 ,  0.222414 ],
       [-1.56664  , -0.0714212,  0.0262   , -0.88242  ,  2.17429  ,
        -0.472384 , -1.03598  ,  1.88348  ,  1.88507  , -2.10331  ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad o

...

('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19325913, -0.19325913],
       [-0.19325913, -0.19325913]]))
('b2:', array([[ 21.6],
       [ 21.6]]))
('intervalle j', 80)
('j', 40)
('x', array([[-1.93886  , -0.0533688, -0.515599 ,  0.924431 , -0.60711  ,
         0.997062 ,  0.0989059,  0.743841 , -1.07655  ,  0.4777   ],
       [ 0.949194 ,  2.08524  ,  0.865439 ,  1.79593  ,  0.799989 ,
        -1.24073  ,  1.9071   , -1.98938  ,  1.41118  , -1.81205  ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.1094

('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19243491, -0.19243491],
       [-0.19243491, -0.19243491]]))
('b2:', array([[ 24.48],
       [ 24.48]]))
('intervalle j', 80)
('j', 56)
('x', array([[  1.00806000e+00,   6.96228000e-02,   3.34808000e-02,
         -9.22037000e-01,  -5.44418000e-01,   1.43122000e+00,
          4.64598000e-01,  -7.74003000e-01,  -1.46667000e+00,
         -1.88072000e-03],
       [  5.24034000e-01,  -8.95115000e-01,  -8.75497000e-01,
         -5.70115000e-01,   1.05797000e+00,  -4.30296000e-01,
         -8.86474000e-01,   2.25652000e+00,   1.99261000e-01,
          1.80005000e+00]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2

('j', 66)
('x', array([[ 0.340571, -0.157333,  0.690126, -0.674289, -0.934679,  0.326834,
        -1.37722 ,  0.288279,  1.17816 , -0.728582],
       [ 1.1715  , -1.14672 , -1.28976 , -1.08732 , -0.535133,  2.05866 ,
         1.0016  ,  0.937985, -0.631931, -0.452856]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19232531, -0.19232531],
       [-0.19232531, -0.19232531]]))
('b2:', array([[ 26.46],
       [ 26.46]]))
('intervalle j', 80)
('j', 67)
('x', array([[-1.4224  ,  1.28626 , -0.198254, -0.710507,  1.08925 , -1.19564 ,
        -0.546662,  0.26

('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad oa', (2L, 10L))
('grad w2', (2L, 2L))
('grad b2', (2L, 1L))
('grad hs', (2L, 10L))
('grad ha', (2L, 10L))
('grad w1', (2L, 2L))
('grad b1', (2L, 1L))
('grad x', (2L, 10L))
('grad w2 el', (2L, 2L))
('grad w1 el', (2L, 2L))
('w1:', array([[-0.10198516, -0.10943951],
       [-0.10198516, -0.10943951]]))
('b1:', array([[-0.30288142],
       [-0.30288142]]))
('w2:', array([[-0.19198954, -0.19198954],
       [-0.19198954, -0.19198954]]))
('b2:', array([[ 28.08],
       [ 28.08]]))
('intervalle j', 80)
('j', 76)
('x', array([[ 0.777167 , -0.849023 , -0.226961 ,  0.216921 ,  1.30664  ,
         0.869888 , -0.0586987, -1.09874  , -0.290934 , -0.274161 ],
       [-0.874401 ,  1.77205  ,  2.03826  , -1.82399  , -0.831853 ,
        -1.87677  ,  1.88345  ,  1.02358  , -2.10828  ,  1.21933  ]]))
('ha', (2L, 10L))
('hs', (2L, 10L))
('w2', (2L, 2L))
('oa', (2L, 10L))
('os', (2L, 10L))
('L', (2L, 10L))
('grad

AttributeError: neural_net instance has no attribute 'plot_function_classifieur'

-6. As a second step, copy your existing implementation to modify it to a new
implementation that will use matrix calculus (instead of a loop) on batches
of size K to improve efficiency. Take the matrix expressions in numpy
derived in the first part, and adapt them for a minibatch of size
K. Show in your report what you have modified (precise the
former and new expressions with the shapes of each matrices).

-7. Compare both implementations (with a loop and with matrix calculus)
to check that they both give the same values for the gradients on the
parameters, first for K = 1, then for K = 10. Display the gradients for
both methods.

In [ ]:
np.array((2,3,4))

-8. Time how long takes an epoch on MNIST (1 epoch = 1 full traversal
through the whole training set) for K = 100 for both versions (loop over
a minibatch and matrix calculus).

-9. Adapt your code to compute the error (proportion of misclassified examples)
on the training set as well as the total loss on the training set during each
epoch of the training procedure, and at the end of each epoch, it computes
the error and average loss on the validation set and the test set. Display
the 6 corresponding figures (error and average loss on train/valid/test),
and write them in a log file.

-10. Train your network on the MNIST dataset. Plot the training/valid/test
curves (error and loss as a function of the epoch number, corresponding
to what you wrote in a file in the last question). Include in your report
the curves obtained using your best hyperparameters, i.e. for which you
obtained your best error on the validation set. We suggest 2 plots : the
first one will plot the error rate (train/valid/test with different colors,
precise which color in a legend) and the other one for the averaged loss
(on train/valid/test). You should be able to get less than 5% test error.
Indicate the values of your best hyperparameters corresponding to the
curves. Bonus points are given for a test error of less that 2%.